# 01 – Data Collection and Preparation

**Goal:**  
Gather and merge FBref Premier League data for wingers (2020/21 – 2024/25).  
This notebook scrapes performance data using `BeautifulSoup` and stores a combined dataset in `data/raw/raw_player_data.csv`.

In [2]:
# Import libraries and packages
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
from io import StringIO
import re
import numpy as np
import unicodedata
from difflib import get_close_matches

In [3]:
# Fetch one season player_id extraction
def get_fbref_standard_with_ids(season: str) -> pd.DataFrame:
    """
    Pull the Premier League 'Standard' player table for a given season (e.g., '2024-2025'),
    flatten headers, remove duplicate in-body header rows, and attach player_id.
    """
    url = f"https://fbref.com/en/comps/9/{season}/stats/players/{season}-Premier-League-Stats"
    headers = {"User-Agent": "Mozilla/5.0"}
    html = requests.get(url, headers=headers, timeout=30).text
    soup = BeautifulSoup(html, "lxml")

    # Find the commented table
    comment = None
    for c in soup.find_all(string=lambda t: isinstance(t, Comment)):
        if 'id="stats_standard"' in c:
            comment = c
            break
    if comment is None:
        raise ValueError(f"stats_standard table not found for {season}")

    # extract player_id from player links
    frag = BeautifulSoup(comment, "lxml")
    tbody = frag.find("tbody")
    row_player_ids = []
    if tbody:
        for tr in tbody.find_all("tr"):
            if "class" in tr.attrs and "thead" in tr.get("class", []):
                continue
            td = tr.find("td", {"data-stat": "player"})
            pid = None
            if td:
                a = td.find("a", href=True)
                if a and a["href"]:
                    m = re.search(r"/en/players/([A-Za-z0-9]+)/", a["href"])
                    if m:
                        pid = m.group(1)
            row_player_ids.append(pid)

    # Read the raw table and flatten
    df = pd.read_html(StringIO(comment), header=[0,1])[0]
    df.columns = [
        "_".join([str(x) for x in col if "Unnamed" not in str(x)]).strip("_")
        for col in df.columns
    ]

    # Remove repeated header rows inside the data
    if "Rk" in df.columns:
        df = df[df["Rk"] != "Rk"].copy()
    df = df.reset_index(drop=True)

    # Attach player_id if lengths match; else leave NaN
    if len(row_player_ids) == len(df):
        df["player_id"] = row_player_ids
    else:
        df["player_id"] = np.nan

    df["season"] = season
    return df

In [4]:
# Collapse to player-season (handle transfers within the season)
def collapse_to_player_season(df: pd.DataFrame) -> pd.DataFrame:
    """
    Collapse (player, squad) rows into a single player-season row:
      - Sum counting stats and minutes across clubs
      - Identify Primary_Squad (most minutes)
      - Create per-90 features using Playing Time_Min
    """
    MIN = "Playing Time_Min" 
    if MIN not in df.columns:
        raise ValueError(f"Column '{MIN}' not found in dataframe.")

    num_cols_force = [
        MIN, "Performance_Gls", "Performance_Ast",
        "Expected_xG", "Expected_xAG", "Expected_npxG", "Expected_npxG+xAG",
        "Progression_PrgC", "Progression_PrgP", "Progression_PrgR",
    ]
    for c in num_cols_force:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df[MIN] = df[MIN].fillna(0)

    # Prefer player_id; fallback to Player if IDs are missing
    if "player_id" in df.columns and df["player_id"].notna().any():
        key_id = "player_id"
    else:
        df["player_key"] = df["Player"].astype(str) + (
            "_" + df["Born"].astype(str) if "Born" in df.columns else ""
        )
        key_id = "player_key"

    # Aggregate minutes + counting stats by (player, season)
    agg_cols = [MIN] + [c for c in df.columns if c in num_cols_force and c != MIN]
    agg = (
        df.groupby([key_id, "season"], as_index=False)[agg_cols]
          .sum(min_count=1)
    )

    # Bring a stable display name
    name_map = (
        df.groupby([key_id, "season"])["Player"]
          .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else s.iloc[0])
          .reset_index()
    )

    # Primary squad = squad with most minutes
    if "Squad" in df.columns:
        prim = (
            df.groupby([key_id, "season", "Squad"], as_index=False)[MIN]
              .sum()
              .sort_values([key_id, "season", MIN], ascending=[True, True, False])
              .drop_duplicates([key_id, "season"])
              .rename(columns={"Squad": "Primary_Squad"})
              [[key_id, "season", "Primary_Squad"]]
        )
    else:
        prim = pd.DataFrame(columns=[key_id, "season", "Primary_Squad"])

    out = agg.merge(name_map, on=[key_id, "season"], how="left").merge(prim, on=[key_id, "season"], how="left")
    out = out.rename(columns={key_id: "player_key_or_id"})

    # Per 90s
    def per90(n, m): 
        return (n / m) * 90 if pd.notna(m) and m > 0 else 0

    per90_map = {
        "Performance_Gls": "gls_per90",
        "Performance_Ast": "ast_per90",
        "Expected_xG":     "xg_per90",
        "Expected_xAG":    "xag_per90",
        "Expected_npxG":   "npxg_per90",
        "Progression_PrgC":"prgC_per90",
        "Progression_PrgP":"prgP_per90",
        "Progression_PrgR":"prgR_per90",
    }
    for src, newc in per90_map.items():
        if src in out.columns:
            out[newc] = out.apply(lambda r: per90(r[src], r[MIN]), axis=1)

    # Tidy ordering
    front = ["player_key_or_id", "Player", "Primary_Squad", "season", MIN]
    per90_cols = [c for c in out.columns if c.endswith("_per90")]
    out = out[front + per90_cols + [c for c in out.columns if c not in front + per90_cols]]

    return out


In [5]:
def one_row_per_player_with_lags(seasons=("2022-2023", "2023-2024", "2024-2025")) -> pd.DataFrame:
    """
    Build one row per player for the latest season, including:
      - numeric lags: *_t1, *_t2, *_roll2 (per90 metrics + minutes)
      - club lags: Primary_Squad_t1, Primary_Squad_t2
    """
    collapsed = []
    for s in seasons:
        raw = get_fbref_standard_with_ids(s)
        collapsed.append(collapse_to_player_season(raw))
    panel = pd.concat(collapsed, ignore_index=True)

    # Sort by player and season
    panel["season_start"] = panel["season"].str.split("-").str[0].astype(int)
    panel = panel.sort_values(["player_key_or_id", "season_start"])

    # Base numeric features = all *_per90 + minutes (if present)
    base_feats = [c for c in panel.columns if c.endswith("_per90")]
    if "Playing Time_Min" in panel.columns:
        base_feats.append("Playing Time_Min")

    # Add numeric and club lags
    def add_lags(g: pd.DataFrame) -> pd.DataFrame:
        for c in base_feats:
            g[c + "_t1"] = g[c].shift(1)
            g[c + "_t2"] = g[c].shift(2)
            g[c + "_roll2"] = g[c].shift(1).rolling(2).mean()
        # club lags
        g["Primary_Squad_t1"] = g["Primary_Squad"].shift(1)
        g["Primary_Squad_t2"] = g["Primary_Squad"].shift(2)
        return g

    panel = panel.groupby("player_key_or_id", group_keys=False).apply(add_lags)

    # Keep only the latest season
    base_season = max(seasons)
    latest = panel[panel["season"] == base_season].copy()

    # Final column order
    id_cols = ["player_key_or_id", "Player", "Primary_Squad", "Primary_Squad_t1", "Primary_Squad_t2", "season"]
    min_cols = [c for c in latest.columns if c.startswith("Playing Time_Min")]
    lagged_cols = [c for c in latest.columns if c.endswith(("_per90", "_t1", "_t2", "_roll2"))]
    cols = [c for c in (id_cols + min_cols + lagged_cols) if c in latest.columns]

    return latest[cols]


In [15]:
def one_row_per_player_with_lags(seasons=("2022-2023", "2023-2024", "2024-2025")) -> pd.DataFrame:
    collapsed = []
    for s in seasons:
        raw = get_fbref_standard_with_ids(s)
        collapsed.append(collapse_to_player_season(raw))
    panel = pd.concat(collapsed, ignore_index=True)

    panel["season_start"] = panel["season"].str.split("-").str[0].astype(int)
    panel = panel.sort_values(["player_key_or_id", "season_start"])

    base_feats = [c for c in panel.columns if c.endswith("_per90")]
    if "Playing Time_Min" in panel.columns:
        base_feats.append("Playing Time_Min")

    def add_lags(g: pd.DataFrame) -> pd.DataFrame:
        for c in base_feats:
            g[c + "_t1"] = g[c].shift(1)
            g[c + "_t2"] = g[c].shift(2)
            g[c + "_roll2"] = g[c].shift(1).rolling(2).mean()
        g["Primary_Squad_t1"] = g["Primary_Squad"].shift(1)
        g["Primary_Squad_t2"] = g["Primary_Squad"].shift(2)
        return g

    panel = panel.groupby("player_key_or_id", group_keys=False).apply(add_lags)

    base_season = max(seasons)
    latest = panel[panel["season"] == base_season].copy()

    # Build column list without overlaps
    id_cols = ["player_key_or_id", "Player", "season", "Primary_Squad", "Primary_Squad_t1", "Primary_Squad_t2"]

    # keep current minutes only
    current_min_cols = ["Playing Time_Min"] if "Playing Time_Min" in latest.columns else []

    # keep current per90s
    current_per90 = [c for c in latest.columns if c.endswith("_per90") and not c.endswith(("_t1","_t2","_roll2"))]

    # numeric lags
    lag_suffixes = ["_t1", "_t2", "_roll2"]
    numeric_lags = []
    for c in base_feats:
        for suf in lag_suffixes:
            col = f"{c}{suf}"
            if col in latest.columns:
                numeric_lags.append(col)

    # character lags
    numeric_lags = [c for c in numeric_lags if not c.startswith("Primary_Squad")]

    cols = id_cols + current_min_cols + current_per90 + numeric_lags
    return latest[cols]


In [16]:
# Pull & collapse a single season
df_2425_raw   = get_fbref_standard_with_ids("2024-2025")
df_2425_season = collapse_to_player_season(df_2425_raw)
df_2425_season.head()

# Build one row per player for 24/25, with 23/24 and 22/23 as lags
one_line = one_row_per_player_with_lags(("2022-2023","2023-2024","2024-2025"))
one_line.head()

/var/folders/82/l14cd08525969c8yj6mfctfh0000gn/T/ipykernel_90220/2400782724.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  panel = panel.groupby("player_key_or_id", group_keys=False).apply(add_lags)


,player_key_or_id,Player,season,Primary_Squad,Primary_Squad_t1,Primary_Squad_t2,Playing Time_Min,gls_per90,ast_per90,xg_per90,...,prgC_per90_roll2,prgP_per90_t1,prgP_per90_t2,prgP_per90_roll2,prgR_per90_t1,prgR_per90_t2,prgR_per90_roll2,Playing Time_Min_t1,Playing Time_Min_t2,Playing Time_Min_roll2
1124,003cf4d1,Jayden Danns,2024-2025,Liverpool,Liverpool,None,11,0.0,0.000000,0.000000,...,NaN,10.000000,NaN,NaN,20.000000,NaN,NaN,9.0,NaN,NaN
1125,007414dd,Samuel Edozie,2024-2025,Southampton,Southampton,None,65,0.0,0.000000,0.000000,...,NaN,2.063492,NaN,NaN,6.190476,NaN,NaN,567.0,NaN,NaN
1126,00953a9d,Mikey Moore,2024-2025,Tottenham,Tottenham,None,366,0.0,0.245902,0.049180,...,NaN,22.500000,NaN,NaN,45.000000,NaN,NaN,4.0,NaN,NaN
1127,00963611,Konstantinos Mavropanos,2024-2025,West Ham,West Ham,None,2045,0.0,0.000000,0.066015,...,NaN,2.280000,NaN,NaN,0.120000,NaN,NaN,1500.0,NaN,NaN
1128,0107757e,Filip Jørgensen,2024-2025,Chelsea,None,None,540,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
one_line.info()

<class 'pandas.core.frame.DataFrame'>
Index: 562 entries, 1124 to 1685
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_key_or_id        562 non-null    object 
 1   Player                  562 non-null    object 
 2   season                  562 non-null    object 
 3   Primary_Squad           562 non-null    object 
 4   Primary_Squad_t1        397 non-null    object 
 5   Primary_Squad_t2        273 non-null    object 
 6   Playing Time_Min        562 non-null    int64  
 7   gls_per90               562 non-null    float64
 8   ast_per90               562 non-null    float64
 9   xg_per90                562 non-null    float64
 10  xag_per90               562 non-null    float64
 11  npxg_per90              562 non-null    float64
 12  prgC_per90              562 non-null    float64
 13  prgP_per90              562 non-null    float64
 14  prgR_per90              562 non-null    flo

In [18]:
# Load Players Position Dataset
positions = pd.read_csv(r"/Users/emmanuel/Documents/Portfolio Projects/FA_Player_Development_Model/data/raw/Players_Position.csv")

In [19]:
# Normalise names for matching
def normalise_name(s: str) -> str:
    if pd.isna(s): return ""
    s = unicodedata.normalize("NFKD", str(s))
    s = "".join(ch for ch in s if not unicodedata.combining(ch))  # remove accents
    s = s.strip().lower().replace("’","'").replace("`","'").replace("´","'")
    return s

In [20]:
# Clean positions headers and find the player column
positions.columns = [c.strip() for c in positions.columns]  # trim whitespace
lower_map = {c.lower(): c for c in positions.columns}

player_col = None
for cand in ["player", "name", "player_name"]:
    if cand in lower_map:
        player_col = lower_map[cand]
        break

if player_col is None:
    raise ValueError(f"Couldn't find a player-name column in positions. Columns are: {positions.columns.tolist()}")

# Rename names
rename_dict = {player_col: "Player"}
if "position" in lower_map: rename_dict[lower_map["position"]] = "Transfermarkt_Position"
if "club" in lower_map:     rename_dict[lower_map["club"]]     = "Transfermarkt_Club"
positions = positions.rename(columns=rename_dict)

In [21]:
# Build normalised keys on both dataframes
if "Player" not in one_line.columns:
    raise ValueError(f"'Player' not found in one_line columns: {one_line.columns.tolist()}")

one_line["player_norm"]  = one_line["Player"].apply(normalise_name)
positions["player_norm"] = positions["Player"].apply(normalise_name)

In [22]:
# Inner join -> keeps ONLY players present in both datasets
keep_cols = ["player_norm", "Player"]
if "Transfermarkt_Position" in positions.columns: keep_cols.append("Transfermarkt_Position")
if "Transfermarkt_Club" in positions.columns:     keep_cols.append("Transfermarkt_Club")

wingers_final = one_line.merge(
    positions[keep_cols].drop_duplicates("player_norm"),
    on="player_norm",
    how="inner",
    suffixes=("", "_tm")
)
wingers_final.head()

,player_key_or_id,Player,season,Primary_Squad,Primary_Squad_t1,Primary_Squad_t2,Playing Time_Min,gls_per90,ast_per90,xg_per90,...,prgR_per90_t1,prgR_per90_t2,prgR_per90_roll2,Playing Time_Min_t1,Playing Time_Min_t2,Playing Time_Min_roll2,player_norm,Player_tm,Transfermarkt_Position,Transfermarkt_Club
0,0cd31129,Brennan Johnson,2024-2025,Tottenham,Tottenham,Nott'ham Forest,2177,0.454754,0.124024,0.429949,...,11.899225,4.754601,8.326913,2322.0,2934.0,2628.0,brennan johnson,Brennan Johnson,Right Winger,Tottenham Hotspur
1,116c35df,Donyell Malen,2024-2025,Aston Villa,None,None,305,0.885246,0.000000,0.649180,...,NaN,NaN,NaN,NaN,NaN,NaN,donyell malen,Donyell Malen,Right Winger,Aston Villa
2,15f3ec41,Callum Hudson-Odoi,2024-2025,Nott'ham Forest,Nott'ham Forest,None,2194,0.205105,0.082042,0.102552,...,8.398058,NaN,NaN,1854.0,NaN,NaN,callum hudson-odoi,Callum Hudson-Odoi,Left Winger,Nottingham Forest
3,1971591f,Cody Gakpo,2024-2025,Liverpool,Liverpool,Liverpool,1935,0.465116,0.186047,0.330233,...,7.095209,6.728395,6.911802,1649.0,1458.0,1553.5,cody gakpo,Cody Gakpo,Left Winger,Liverpool
4,2bd83368,Anthony Gordon,2024-2025,Newcastle Utd,Newcastle Utd,Everton,2434,0.221857,0.184881,0.295809,...,7.224913,6.371681,6.798297,2890.0,1582.0,2236.0,anthony gordon,Anthony Gordon,Left Winger,Newcastle United


In [23]:
df = wingers_final.copy().reset_index(drop=True)

curr = np.asarray(df['Playing Time_Min'])
t1   = np.asarray(df['Playing Time_Min_t1'])
t2   = np.asarray(df['Playing Time_Min_t2'])

m = (curr >= 500) & (t1 >= 500) & (t2 >= 500)
df_3seasons_500 = df[m].copy()

print("Rows with 3 seasons and ≥500 each:", len(df_3seasons_500))
df_3seasons_500.head()


Rows with 3 seasons and ≥500 each: 22


,player_key_or_id,Player,season,Primary_Squad,Primary_Squad_t1,Primary_Squad_t2,Playing Time_Min,gls_per90,ast_per90,xg_per90,...,prgR_per90_t1,prgR_per90_t2,prgR_per90_roll2,Playing Time_Min_t1,Playing Time_Min_t2,Playing Time_Min_roll2,player_norm,Player_tm,Transfermarkt_Position,Transfermarkt_Club
0,0cd31129,Brennan Johnson,2024-2025,Tottenham,Tottenham,Nott'ham Forest,2177,0.454754,0.124024,0.429949,...,11.899225,4.754601,8.326913,2322.0,2934.0,2628.0,brennan johnson,Brennan Johnson,Right Winger,Tottenham Hotspur
3,1971591f,Cody Gakpo,2024-2025,Liverpool,Liverpool,Liverpool,1935,0.465116,0.186047,0.330233,...,7.095209,6.728395,6.911802,1649.0,1458.0,1553.5,cody gakpo,Cody Gakpo,Left Winger,Liverpool
4,2bd83368,Anthony Gordon,2024-2025,Newcastle Utd,Newcastle Utd,Everton,2434,0.221857,0.184881,0.295809,...,7.224913,6.371681,6.798297,2890.0,1582.0,2236.0,anthony gordon,Anthony Gordon,Left Winger,Newcastle United
5,2f9e4435,Dango Ouattara,2024-2025,Bournemouth,Bournemouth,Bournemouth,2005,0.314214,0.179551,0.377057,...,6.590724,6.309428,6.450076,1229.0,1241.0,1235.0,dango ouattara,Dango Ouattara,Left Winger,Brentford
8,38ceb24a,Leandro Trossard,2024-2025,Arsenal,Arsenal,Brighton,2546,0.282797,0.247447,0.254517,...,8.459673,8.815672,8.637672,1649.0,2246.0,1947.5,leandro trossard,Leandro Trossard,Left Winger,Arsenal


In [25]:
# Extracting Dataset
df_3seasons_500.to_csv("/Users/emmanuel/Documents/Portfolio Projects/FA_Player_Development_Model/data/processed/Wingers.csv", index=True)